In [1]:
import os
os.chdir("../")

In [2]:
%pwd

'/Volumes/Personal/projects/vision/Image-classification-DL'

In [3]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Spandan-Roy/Image-classification-DL.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Spandan-Roy"
os.environ["MLFLOW_TRACKING_PASSWORD"]="937eb325a30503129b948a909925fa666b3e0ec8"

In [4]:
import tensorflow as tf
model = tf.keras.models.load_model("artifacts/training.model.h5")

2024-12-20 19:17:42.674525: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4 Pro
2024-12-20 19:17:42.674547: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 24.00 GB
2024-12-20 19:17:42.674552: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 8.00 GB
2024-12-20 19:17:42.674742: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-12-20 19:17:42.674755: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data:Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int
    

In [6]:
from CNN_Classifier.constants import *
from CNN_Classifier.utils.common import read_yaml,create_directories,save_json



In [7]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training.model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/Spandan-Roy/Image-classification-DL.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [8]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.sklearn
from urllib.parse import urlparse
from mlflow.models import infer_signature

In [9]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()
    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(self.model,artifact_path= "model",registered_model_name="VG16")
            else:
                mlflow.sklearn.log_model(self.model,artifact_path= "model",registered_model_name="VG16")

In [10]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-12-20 19:17:49,247: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-12-20 19:17:49,249: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-20 19:17:49,250: INFO: common: created directory at: artifacts]
Found 3732 images belonging to 4 classes.


2024-12-20 19:17:49.736127: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


38/38 [==============================] - 18s 441ms/step - loss: 4.7633 - accuracy: 0.7567
[2024-12-20 19:18:07,257: INFO: common: json file saved at: scores.json]


2024/12/20 19:18:11 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /var/folders/3t/wph0jhys56g0cdp9y_c2vb3r0000gn/T/tmp2bozfpwz/model/model.pkl, flavor: sklearn), fall back to return ['scikit-learn==1.6.0', 'cloudpickle==2.2.1']. Set logging level to DEBUG to see the full traceback.
/Users/spandy/miniforge3/envs/vision/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Registered model 'VG16' already exists. Creating a new version of this model...
2024/12/20 19:18:31 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: VG16, version 4
Created version '4' of model 'VG16'.
